In [16]:
# Importando bibliotecas
import requests    # Para fazer a conexao com a pagina
import re    # Para utilizar expressoes regulares 
import pandas as pd    # Para trabalhar com tabelas
import numpy as np
import wget    
import zipfile
import csv
import codecs
import os

from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import HTML

In [2]:
spark = SparkSession.builder.appName('Dataframe').getOrCreate()
spark

In [3]:
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [3]:
"""
estado = ['AC','AL','AM','AP','BA','CE','DF','ES','GO','MA','MG','MS','MT','PA','PB','PE','PI','PR','RJ','RN','RO','RR','RS','SC','SE','SP','TO']
"""

"\nestado = ['AC','AL','AM','AP','BA','CE','DF','ES','GO','MA','MG','MS','MT','PA','PB','PE','PI','PR','RJ','RN','RO','RR','RS','SC','SE','SP','TO']\n"

In [4]:
"""
mes = ['01','02','03','04','05','06','07','08','09','10','11','12']

"""

"\nmes = ['01','02','03','04','05','06','07','08','09','10','11','12']\n\n"

In [5]:
"""
tp = ['CONS', 'DET']

"""

"\ntp = ['CONS', 'DET']\n\n"

In [6]:
"""

for t in tp:
    with open(f'Tab{t}.csv', 'w', encoding="utf-8", newline='') as Tabell:
        escrever = csv.writer(Tabell)
        indi = 1
        for est in estado:
            for me in mes:
                pagina = f"http://ftp.dadosabertos.ans.gov.br/FTP/PDA/TISS/HOSPITALAR/2019/{est}/{est}_2019{me}_HOSP_{t}.zip"
                arquivo = wget.download(pagina, bar = None)
                try:
                    with zipfile.ZipFile(arquivo) as zipado:
                        arqcsv = str(arquivo)[0:-3]+'csv'
                        with zipado.open(arqcsv,mode='r') as tabelacsv:
                            for linha in csv.DictReader(codecs.iterdecode(tabelacsv, 'utf-8'), delimiter=','):
                                indice = list(linha.keys())[0].split(';')
                                valor = list(linha.values())[0].split(';')
                                if indi:
                                    escrever.writerow(indice)
                                    indi = 0
                                escrever.writerow(valor)
                    
                except:
                    print(f"Erro no arquivo {str(arquivo)}")
                    os.remove(arquivo)
                    break
                os.remove(arquivo)
else:
    TabCons = pd.read_csv('TabCONS.csv')
    TabDet = pd.read_csv('TabDET.csv')
    comb = pd.merge(TabCons, TabDet, how = 'inner', on = 'ID_EVENTO_ATENCAO_SAUDE')
    comb.to_csv('UniTabela.csv', encoding = "utf-8", index = False)
    
"""

'\n\nfor t in tp:\n    with open(f\'Tab{t}.csv\', \'w\', encoding="utf-8", newline=\'\') as Tabell:\n        escrever = csv.writer(Tabell)\n        indi = 1\n        for est in estado:\n            for me in mes:\n                pagina = f"http://ftp.dadosabertos.ans.gov.br/FTP/PDA/TISS/HOSPITALAR/2019/{est}/{est}_2019{me}_HOSP_{t}.zip"\n                arquivo = wget.download(pagina, bar = None)\n                try:\n                    with zipfile.ZipFile(arquivo) as zipado:\n                        arqcsv = str(arquivo)[0:-3]+\'csv\'\n                        with zipado.open(arqcsv,mode=\'r\') as tabelacsv:\n                            for linha in csv.DictReader(codecs.iterdecode(tabelacsv, \'utf-8\'), delimiter=\',\'):\n                                indice = list(linha.keys())[0].split(\';\')\n                                valor = list(linha.values())[0].split(\';\')\n                                if indi:\n                                    escrever.writerow(indice)\n  

In [3]:
dadosdet = spark.read.csv('TabDET.csv',header=True, inferSchema=True, sep=',')
dadoscons = spark.read.csv('TabCONS.csv',header=True, inferSchema=True, sep=',')

In [8]:
dadosdet.printSchema()

root
 |-- ID_EVENTO_ATENCAO_SAUDE: long (nullable = true)
 |-- UF_PRESTADOR: string (nullable = true)
 |-- TEMPO_DE_PERMANENCIA: double (nullable = true)
 |-- ANO_MES_EVENTO: string (nullable = true)
 |-- CD_PROCEDIMENTO: string (nullable = true)
 |-- CD_TABELA_REFERENCIA: integer (nullable = true)
 |-- QT_ITEM_EVENTO_INFORMADO: integer (nullable = true)
 |-- VL_ITEM_EVENTO_INFORMADO: integer (nullable = true)
 |-- VL_ITEM_PAGO_FORNECEDOR: integer (nullable = true)
 |-- IND_PACOTE: integer (nullable = true)
 |-- IND_TABELA_PROPRIA: integer (nullable = true)



In [9]:
dadoscons.printSchema()

root
 |-- ID_EVENTO_ATENCAO_SAUDE: long (nullable = true)
 |-- ID_PLANO: integer (nullable = true)
 |-- FAIXA_ETARIA: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- CD_MUNICIPIO_BENEFICIARIO: integer (nullable = true)
 |-- PORTE: string (nullable = true)
 |-- CD_MODALIDADE: integer (nullable = true)
 |-- NM_MODALIDADE: string (nullable = true)
 |-- CD_MUNICIPIO_PRESTADOR: integer (nullable = true)
 |-- UF_PRESTADOR: string (nullable = true)
 |-- TEMPO_DE_PERMANENCIA: integer (nullable = true)
 |-- ANO_MES_EVENTO: string (nullable = true)
 |-- CD_CARATER_ATENDIMENTO: integer (nullable = true)
 |-- CD_TIPO_INTERNACAO: integer (nullable = true)
 |-- CD_REGIME_INTERNACAO: integer (nullable = true)
 |-- CD_MOTIVO_SAIDA: integer (nullable = true)
 |-- CID_1: string (nullable = true)
 |-- CID_2: string (nullable = true)
 |-- CID_3: string (nullable = true)
 |-- CID_4: string (nullable = true)
 |-- QT_DIARIA_ACOMPANHANTE: integer (nullable = true)
 |-- QT_DIARIA_UTI: integer

In [10]:
dadosdet.head(5)

[Row(ID_EVENTO_ATENCAO_SAUDE=2360654932, UF_PRESTADOR='AC', TEMPO_DE_PERMANENCIA=27.0, ANO_MES_EVENTO='2019-01', CD_PROCEDIMENTO='40307026', CD_TABELA_REFERENCIA=22, QT_ITEM_EVENTO_INFORMADO=1, VL_ITEM_EVENTO_INFORMADO=96, VL_ITEM_PAGO_FORNECEDOR=0, IND_PACOTE=0, IND_TABELA_PROPRIA=0),
 Row(ID_EVENTO_ATENCAO_SAUDE=2360654932, UF_PRESTADOR='AC', TEMPO_DE_PERMANENCIA=27.0, ANO_MES_EVENTO='2019-01', CD_PROCEDIMENTO='40307182', CD_TABELA_REFERENCIA=22, QT_ITEM_EVENTO_INFORMADO=1, VL_ITEM_EVENTO_INFORMADO=48, VL_ITEM_PAGO_FORNECEDOR=0, IND_PACOTE=0, IND_TABELA_PROPRIA=0),
 Row(ID_EVENTO_ATENCAO_SAUDE=2360654932, UF_PRESTADOR='AC', TEMPO_DE_PERMANENCIA=27.0, ANO_MES_EVENTO='2019-01', CD_PROCEDIMENTO='40307212', CD_TABELA_REFERENCIA=22, QT_ITEM_EVENTO_INFORMADO=1, VL_ITEM_EVENTO_INFORMADO=30, VL_ITEM_PAGO_FORNECEDOR=0, IND_PACOTE=0, IND_TABELA_PROPRIA=0),
 Row(ID_EVENTO_ATENCAO_SAUDE=2360654932, UF_PRESTADOR='AC', TEMPO_DE_PERMANENCIA=27.0, ANO_MES_EVENTO='2019-01', CD_PROCEDIMENTO='40307760'

In [11]:
dadoscons.show(5)

+-----------------------+--------+------------+---------+-------------------------+-----+-------------+------------------+----------------------+------------+--------------------+--------------+----------------------+------------------+--------------------+---------------+-----+-----+-----+-----+----------------------+-------------+-------------------+------------------------+
|ID_EVENTO_ATENCAO_SAUDE|ID_PLANO|FAIXA_ETARIA|     SEXO|CD_MUNICIPIO_BENEFICIARIO|PORTE|CD_MODALIDADE|     NM_MODALIDADE|CD_MUNICIPIO_PRESTADOR|UF_PRESTADOR|TEMPO_DE_PERMANENCIA|ANO_MES_EVENTO|CD_CARATER_ATENDIMENTO|CD_TIPO_INTERNACAO|CD_REGIME_INTERNACAO|CD_MOTIVO_SAIDA|CID_1|CID_2|CID_3|CID_4|QT_DIARIA_ACOMPANHANTE|QT_DIARIA_UTI|IND_ACIDENTE_DOENCA|LG_VALOR_PREESTABELECIDO|
+-----------------------+--------+------------+---------+-------------------------+-----+-------------+------------------+----------------------+------------+--------------------+--------------+----------------------+------------------+----

In [12]:
dadosdet.count(), dadoscons.count()

39926392

In [14]:
dadosdet.filter('ID_EVENTO_ATENCAO_SAUDE = 2289170847').show()

+-----------------------+------------+--------------------+--------------+---------------+--------------------+------------------------+------------------------+-----------------------+----------+------------------+
|ID_EVENTO_ATENCAO_SAUDE|UF_PRESTADOR|TEMPO_DE_PERMANENCIA|ANO_MES_EVENTO|CD_PROCEDIMENTO|CD_TABELA_REFERENCIA|QT_ITEM_EVENTO_INFORMADO|VL_ITEM_EVENTO_INFORMADO|VL_ITEM_PAGO_FORNECEDOR|IND_PACOTE|IND_TABELA_PROPRIA|
+-----------------------+------------+--------------------+--------------+---------------+--------------------+------------------------+------------------------+-----------------------+----------+------------------+
|             2289170847|          AC|                 1.0|       2019-01| TABELA PRÓPRIA|                   0|                      47|                    2333|                   null|      null|              null|
|             2289170847|          AC|                 1.0|       2019-01|            030|                  63|                     124|

In [15]:
dadoscons.filter('ID_EVENTO_ATENCAO_SAUDE = 2289170847').show()

+-----------------------+--------+------------+---------+-------------------------+-----+-------------+------------------+----------------------+------------+--------------------+--------------+----------------------+------------------+--------------------+---------------+-----+-----+-----+-----+----------------------+-------------+-------------------+------------------------+
|ID_EVENTO_ATENCAO_SAUDE|ID_PLANO|FAIXA_ETARIA|     SEXO|CD_MUNICIPIO_BENEFICIARIO|PORTE|CD_MODALIDADE|     NM_MODALIDADE|CD_MUNICIPIO_PRESTADOR|UF_PRESTADOR|TEMPO_DE_PERMANENCIA|ANO_MES_EVENTO|CD_CARATER_ATENDIMENTO|CD_TIPO_INTERNACAO|CD_REGIME_INTERNACAO|CD_MOTIVO_SAIDA|CID_1|CID_2|CID_3|CID_4|QT_DIARIA_ACOMPANHANTE|QT_DIARIA_UTI|IND_ACIDENTE_DOENCA|LG_VALOR_PREESTABELECIDO|
+-----------------------+--------+------------+---------+-------------------------+-----+-------------+------------------+----------------------+------------+--------------------+--------------+----------------------+------------------+----

In [16]:
len(dadosdet.columns) , len(dadoscons.columns)

(11, 24)

In [17]:
for i in dadoscons.columns:
    if i in dadosdet.columns: print(i)

ID_EVENTO_ATENCAO_SAUDE
UF_PRESTADOR
TEMPO_DE_PERMANENCIA
ANO_MES_EVENTO


In [4]:
dados = dadoscons.join(dadosdet, ["ID_EVENTO_ATENCAO_SAUDE", "UF_PRESTADOR", "TEMPO_DE_PERMANENCIA", "ANO_MES_EVENTO"] , "full")
del dadosdet
del dadoscons

In [21]:
dados.filter('ID_EVENTO_ATENCAO_SAUDE = 2289170847').show()

+-----------------------+------------+--------------------+--------------+--------+------------+---------+-------------------------+-----+-------------+------------------+----------------------+----------------------+------------------+--------------------+---------------+-----+-----+-----+-----+----------------------+-------------+-------------------+------------------------+---------------+--------------------+------------------------+------------------------+-----------------------+----------+------------------+
|ID_EVENTO_ATENCAO_SAUDE|UF_PRESTADOR|TEMPO_DE_PERMANENCIA|ANO_MES_EVENTO|ID_PLANO|FAIXA_ETARIA|     SEXO|CD_MUNICIPIO_BENEFICIARIO|PORTE|CD_MODALIDADE|     NM_MODALIDADE|CD_MUNICIPIO_PRESTADOR|CD_CARATER_ATENDIMENTO|CD_TIPO_INTERNACAO|CD_REGIME_INTERNACAO|CD_MOTIVO_SAIDA|CID_1|CID_2|CID_3|CID_4|QT_DIARIA_ACOMPANHANTE|QT_DIARIA_UTI|IND_ACIDENTE_DOENCA|LG_VALOR_PREESTABELECIDO|CD_PROCEDIMENTO|CD_TABELA_REFERENCIA|QT_ITEM_EVENTO_INFORMADO|VL_ITEM_EVENTO_INFORMADO|VL_ITEM_PAGO_

In [5]:
#Exportando somente para nao precisar realizar a extração do conteudo novamente.
dados.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/TabCOMP.csv')

<br><br><br>

In [4]:
dados = spark.read.csv('TabCOMP',header=True, inferSchema=True, sep=',')

In [5]:
print(dados.count())
print(len(dados.columns))

39943332
31


In [6]:
dados.show(5)

+-----------------------+------------+--------------------+--------------+--------+------------+---------+-------------------------+-----+-------------+------------------+----------------------+----------------------+------------------+--------------------+---------------+-----+-----+-----+-----+----------------------+-------------+-------------------+------------------------+---------------+--------------------+------------------------+------------------------+-----------------------+----------+------------------+
|ID_EVENTO_ATENCAO_SAUDE|UF_PRESTADOR|TEMPO_DE_PERMANENCIA|ANO_MES_EVENTO|ID_PLANO|FAIXA_ETARIA|     SEXO|CD_MUNICIPIO_BENEFICIARIO|PORTE|CD_MODALIDADE|     NM_MODALIDADE|CD_MUNICIPIO_PRESTADOR|CD_CARATER_ATENDIMENTO|CD_TIPO_INTERNACAO|CD_REGIME_INTERNACAO|CD_MOTIVO_SAIDA|CID_1|CID_2|CID_3|CID_4|QT_DIARIA_ACOMPANHANTE|QT_DIARIA_UTI|IND_ACIDENTE_DOENCA|LG_VALOR_PREESTABELECIDO|CD_PROCEDIMENTO|CD_TABELA_REFERENCIA|QT_ITEM_EVENTO_INFORMADO|VL_ITEM_EVENTO_INFORMADO|VL_ITEM_PAGO_

In [7]:
dados.groupBy(dados.columns).count().filter("count > 1").drop('count').show()

+-----------------------+------------+--------------------+--------------+--------+------------+---------+-------------------------+-----+-------------+-------------+----------------------+----------------------+------------------+--------------------+---------------+-----+-----+-----+-----+----------------------+-------------+-------------------+------------------------+---------------+--------------------+------------------------+------------------------+-----------------------+----------+------------------+
|ID_EVENTO_ATENCAO_SAUDE|UF_PRESTADOR|TEMPO_DE_PERMANENCIA|ANO_MES_EVENTO|ID_PLANO|FAIXA_ETARIA|     SEXO|CD_MUNICIPIO_BENEFICIARIO|PORTE|CD_MODALIDADE|NM_MODALIDADE|CD_MUNICIPIO_PRESTADOR|CD_CARATER_ATENDIMENTO|CD_TIPO_INTERNACAO|CD_REGIME_INTERNACAO|CD_MOTIVO_SAIDA|CID_1|CID_2|CID_3|CID_4|QT_DIARIA_ACOMPANHANTE|QT_DIARIA_UTI|IND_ACIDENTE_DOENCA|LG_VALOR_PREESTABELECIDO|CD_PROCEDIMENTO|CD_TABELA_REFERENCIA|QT_ITEM_EVENTO_INFORMADO|VL_ITEM_EVENTO_INFORMADO|VL_ITEM_PAGO_FORNECEDOR

In [8]:
# FAZER ESCLUIR PERMANTENTE
dados = dados.dropDuplicates()

In [9]:
# Tem que retornar vazio
dados.groupBy(dados.columns).count().filter("count > 1").drop('count').show()

+-----------------------+------------+--------------------+--------------+--------+------------+----+-------------------------+-----+-------------+-------------+----------------------+----------------------+------------------+--------------------+---------------+-----+-----+-----+-----+----------------------+-------------+-------------------+------------------------+---------------+--------------------+------------------------+------------------------+-----------------------+----------+------------------+
|ID_EVENTO_ATENCAO_SAUDE|UF_PRESTADOR|TEMPO_DE_PERMANENCIA|ANO_MES_EVENTO|ID_PLANO|FAIXA_ETARIA|SEXO|CD_MUNICIPIO_BENEFICIARIO|PORTE|CD_MODALIDADE|NM_MODALIDADE|CD_MUNICIPIO_PRESTADOR|CD_CARATER_ATENDIMENTO|CD_TIPO_INTERNACAO|CD_REGIME_INTERNACAO|CD_MOTIVO_SAIDA|CID_1|CID_2|CID_3|CID_4|QT_DIARIA_ACOMPANHANTE|QT_DIARIA_UTI|IND_ACIDENTE_DOENCA|LG_VALOR_PREESTABELECIDO|CD_PROCEDIMENTO|CD_TABELA_REFERENCIA|QT_ITEM_EVENTO_INFORMADO|VL_ITEM_EVENTO_INFORMADO|VL_ITEM_PAGO_FORNECEDOR|IND_PACOT

In [ ]:
from pyspark.sql.functions import col,isnan, when, count

In [17]:
dados.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in dados.columns]).show()

+-----------------------+------------+--------------------+--------------+--------+------------+------+-------------------------+------+-------------+-------------+----------------------+----------------------+------------------+--------------------+---------------+--------+--------+--------+--------+----------------------+-------------+-------------------+------------------------+---------------+--------------------+------------------------+------------------------+-----------------------+----------+------------------+
|ID_EVENTO_ATENCAO_SAUDE|UF_PRESTADOR|TEMPO_DE_PERMANENCIA|ANO_MES_EVENTO|ID_PLANO|FAIXA_ETARIA|  SEXO|CD_MUNICIPIO_BENEFICIARIO| PORTE|CD_MODALIDADE|NM_MODALIDADE|CD_MUNICIPIO_PRESTADOR|CD_CARATER_ATENDIMENTO|CD_TIPO_INTERNACAO|CD_REGIME_INTERNACAO|CD_MOTIVO_SAIDA|   CID_1|   CID_2|   CID_3|   CID_4|QT_DIARIA_ACOMPANHANTE|QT_DIARIA_UTI|IND_ACIDENTE_DOENCA|LG_VALOR_PREESTABELECIDO|CD_PROCEDIMENTO|CD_TABELA_REFERENCIA|QT_ITEM_EVENTO_INFORMADO|VL_ITEM_EVENTO_INFORMADO|VL_